In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
predict = pd.read_csv("test.csv")

#train = train[0:100]
#predict = predict[0:100]

In [ ]:
def handle_nan(data):
    data2 = data.fillna(" ")
    return data2

In [ ]:
import re
import string

def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)  
    return text

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#import nltk
#nltk.download('stopwords')

stop_words = set(stopwords.words("english"))
ps = PorterStemmer()

def stemming(text):
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [ ]:
train = handle_nan(train)
predict = handle_nan(predict)

x = train['text'].apply(wordopt)
y = train['label']
p = predict['text'].apply(wordopt)

#x = train['text'].apply(stemming)
#p = predict['text'].apply(stemming)

# SKlearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

vectorization = CountVectorizer()
x_train = vectorization.fit_transform(x_train)
x_test = vectorization.transform(x_test)

In [ ]:
def manual_testing(model_name, news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_x_test = new_def_test["text"]
    new_x_test = vectorization.transform(new_x_test)
    return "proba of 0 & 1:", model_name.predict_proba(new_x_test)

def preciction(model_name, data):
    new_def_test = p
    new_x_test = vectorization.transform(new_def_test)

    predictions_test = pd.DataFrame(model_name.predict(new_x_test))
    predictions_test_predict_proba = pd.DataFrame(model_name.predict_proba(new_x_test))
    test_id = pd.DataFrame(predict["id"])

    submission = pd.concat([test_id, predictions_test, predictions_test_predict_proba],axis=1)
    submission.columns = ["id",model_name,"proba_0","proba_1"]
    submission.to_csv("ML_Combined_{}.csv".format(model_name),index=False)

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
def cross_val(model_name):
    scores = cross_val_score(model_name, x_train,y_train, scoring="neg_mean_squared_error", cv=2)
    tree_rmse_scores = np.sqrt(-scores)
    return ("Scores:", scores),("Mean:", scores.mean()),("Standard deviation:", scores.std())

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

def Confusion_Matrix_f1_score(model_name):    
    y_train_pred = cross_val_predict(model_name, x_train,y_train, cv=3)
    return confusion_matrix(y_train,y_train_pred), f1_score(y_train,y_train_pred)

## Try different sklearn algos

In [ ]:
from sklearn.naive_bayes import MultinomialNB
Bayesian = MultinomialNB()
Bayesian.fit(x_train,y_train)
Bayesian.score(x_test, y_test)
Confusion_Matrix_f1_score(Bayesian)

In [ ]:
preciction(Bayesian,p)
manual_testing(Bayesian, str("Not quite all the action is  . Peter Thiel, a founder of PayPal and Palantir who was the first outside investor in Facebook, spoke at the Republican convention in July. The New York Times reported on Saturday that Mr. Thiel is giving $1. 25 million to support Mr. Trump’s candidacy even as other supporters flee. (He also recently gave $1 million to a “super PAC” that supports Senator Rob Portman, the Republican freshman running for   in Ohio.) Getting involved in politics used to be seen as clashing with Silicon Valley’s value system: You transform the world by making problems obsolete, not solving them through Washington. Nor did entrepreneurs want to alienate whatever segment of customers did not agree with them politically. Such reticence is no longer in style here. “We’re a bunch of nerds not used to having a lot of limelight,” said Dave McClure, an investor who runs a tech incubator called 500 Startups."))

In [ ]:
from sklearn.linear_model import LogisticRegression

Logistic = LogisticRegression()
Logistic.fit(x_train,y_train)
Confusion_Matrix_f1_score(Logistic)

In [ ]:
preciction(Logistic,p)
manual_testing(Logistic, str("Not quite all the action is  . Peter Thiel, a founder of PayPal and Palantir who was the first outside investor in Facebook, spoke at the Republican convention in July. The New York Times reported on Saturday that Mr. Thiel is giving $1. 25 million to support Mr. Trump’s candidacy even as other supporters flee. (He also recently gave $1 million to a “super PAC” that supports Senator Rob Portman, the Republican freshman running for   in Ohio.) Getting involved in politics used to be seen as clashing with Silicon Valley’s value system: You transform the world by making problems obsolete, not solving them through Washington. Nor did entrepreneurs want to alienate whatever segment of customers did not agree with them politically. Such reticence is no longer in style here. “We’re a bunch of nerds not used to having a lot of limelight,” said Dave McClure, an investor who runs a tech incubator called 500 Startups."))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DecisionTree = DecisionTreeClassifier()
DecisionTree.fit(x_train,y_train)
Confusion_Matrix_f1_score(DecisionTree)

In [ ]:
preciction(DecisionTree,p)
manual_testing(DecisionTree, str("Not quite all the action is  . Peter Thiel, a founder of PayPal and Palantir who was the first outside investor in Facebook, spoke at the Republican convention in July. The New York Times reported on Saturday that Mr. Thiel is giving $1. 25 million to support Mr. Trump’s candidacy even as other supporters flee. (He also recently gave $1 million to a “super PAC” that supports Senator Rob Portman, the Republican freshman running for   in Ohio.) Getting involved in politics used to be seen as clashing with Silicon Valley’s value system: You transform the world by making problems obsolete, not solving them through Washington. Nor did entrepreneurs want to alienate whatever segment of customers did not agree with them politically. Such reticence is no longer in style here. “We’re a bunch of nerds not used to having a lot of limelight,” said Dave McClure, an investor who runs a tech incubator called 500 Startups."))

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
k_neighbor = KNeighborsClassifier()
k_neighbor.fit(x_train,y_train)
Confusion_Matrix_f1_score(k_neighbor)

In [ ]:
preciction(k_neighbor,p)
manual_testing(k_neighbor, str("Not quite all the action is  . Peter Thiel, a founder of PayPal and Palantir who was the first outside investor in Facebook, spoke at the Republican convention in July. The New York Times reported on Saturday that Mr. Thiel is giving $1. 25 million to support Mr. Trump’s candidacy even as other supporters flee. (He also recently gave $1 million to a “super PAC” that supports Senator Rob Portman, the Republican freshman running for   in Ohio.) Getting involved in politics used to be seen as clashing with Silicon Valley’s value system: You transform the world by making problems obsolete, not solving them through Washington. Nor did entrepreneurs want to alienate whatever segment of customers did not agree with them politically. Such reticence is no longer in style here. “We’re a bunch of nerds not used to having a lot of limelight,” said Dave McClure, an investor who runs a tech incubator called 500 Startups."))

In [ ]:
#14 min to run
from sklearn.svm import SVC
SVC1 = SVC()
SVC1.fit(x_train,y_train)
Confusion_Matrix_f1_score(SVC1)

from sklearn.ensemble import RandomForestRegressor
RandomForest = RandomForestRegressor(n_estimators=500, max_leaf_nodes=16, n_jobs=3, verbose=True)
RandomForest.fit(x_train,y_train)
Confusion_Matrix_f1_score(RandomForest)

In [ ]:
#xx min to run
from sklearn.neural_network import MLPClassifier
mlp  = MLPClassifier(solver='lbfgs', activation='logistic')
mlp.fit(x_train,y_train)
Confusion_Matrix_f1_score(mlp)

# TF

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd


#Deep learning libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
def manual_predict(model_name, str):
    data = []
    data.append(pred)
    p2 = pd.DataFrame(data)

    p2 = p2[0].apply(wordopt)

    tokenizer_p = Tokenizer(num_words = voc_size)
    tokenizer_p.fit_on_texts(p2)

    p2 = tokenizer_p.texts_to_sequences(p2)

    p2 = tf.keras.preprocessing.sequence.pad_sequences(p2, padding='post', maxlen=sent_length)

    return model_name.predict(p2)

In [ ]:
def preciction_2(model_name, data):
    predictions_test = pd.DataFrame(model_name.predict(data))
    test_id = pd.DataFrame(predict["id"])

    submission = pd.concat([test_id, predictions_test],axis=1)
    submission.columns = ["id",model_name]
    submission.to_csv("ML_Combined_tensorout.csv",index=False)

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x, y, test_size=0.2)

voc_size = 5000

x_train_2=[one_hot(words,voc_size)for words in x_train_1] 
x_test_2=[one_hot(words,voc_size)for words in x_test_1] 
p_2=[one_hot(words,voc_size)for words in p] 


#tokenizer = Tokenizer(num_words=voc_size)
#tokenizer.fit_on_texts(x_train_1)
#
#x_train_2 = tokenizer.texts_to_sequences(x_train_1)
#x_test_2 = tokenizer.texts_to_sequences(x_test_1)
#
#tokenizer_p = Tokenizer(num_words=voc_size)
#tokenizer_p.fit_on_texts(p)
#p_2 = tokenizer_p.texts_to_sequences(p)

sent_length=5000
x_train_3 = tf.keras.preprocessing.sequence.pad_sequences(x_train_2, padding='post', maxlen=sent_length)
x_test_3 = tf.keras.preprocessing.sequence.pad_sequences(x_test_2, padding='post', maxlen=sent_length)
p_3 = tf.keras.preprocessing.sequence.pad_sequences(p_2, padding='post', maxlen=sent_length)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(voc_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit(x_train_3, y_train_1, epochs=50,validation_split=0.1, batch_size=64, shuffle=True, callbacks=[early_stop])

In [ ]:
#test loss, test acc
model.evaluate(x_test_3, y_test_1)

In [ ]:
pred = str(

    "Not quite all the action is  . Peter Thiel, a founder of PayPal and Palantir who was the first outside investor in Facebook, spoke at the Republican convention in July. The New York Times reported on Saturday that Mr. Thiel is giving $1. 25 million to support Mr. Trump’s candidacy even as other supporters flee. (He also recently gave $1 million to a “super PAC” that supports Senator Rob Portman, the Republican freshman running for   in Ohio.) Getting involved in politics used to be seen as clashing with Silicon Valley’s value system: You transform the world by making problems obsolete, not solving them through Washington. Nor did entrepreneurs want to alienate whatever segment of customers did not agree with them politically. Such reticence is no longer in style here. “We’re a bunch of nerds not used to having a lot of limelight,” said Dave McClure, an investor who runs a tech incubator called 500 Startups."
    
    )

In [ ]:
manual_predict(model, pred)

In [ ]:
preciction_2(model, p_3)